In [21]:
import numpy as np
import pandas as pd
import difflib
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import os
f = "movies.csv"
os.path.exists(f)

True

In [3]:
movie_dataset = pd.read_csv("movies.csv")
movie_dataset.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movie_dataset.shape

(4803, 24)

In [44]:
features_chosen = ['genres','keywords','vote_average','cast','director']
print(features_chosen)

['genres', 'keywords', 'vote_average', 'cast', 'director']


In [6]:
for ftr in features_chosen:
    movie_dataset[ftr] = movie_dataset[ftr].fillna('')

In [45]:
ftr_for_recommendation = movie_dataset['genres'] + ' ' + movie_dataset['keywords'] + ' ' + movie_dataset['cast'] + ' ' + movie_dataset['director'] + ' ' + str(movie_dataset['vote_average'])
print(ftr_for_recommendation)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  Edward Burns Kerry Bish\u00e9 ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      Daniel Henney Eliza Coupe Bill Paxton Alan R...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [46]:
vectorizer = TfidfVectorizer()
ftr_vector = vectorizer.fit_transform(ftr_for_recommendation)
print(ftr_vector)

  (0, 4678)	0.03488465043158973
  (0, 3855)	0.03488465043158973
  (0, 25)	0.03488465043158973
  (0, 7684)	0.03488465043158973
  (0, 14172)	0.03488465043158973
  (0, 9332)	0.03488465043158973
  (0, 24)	0.03488465043158973
  (0, 23)	0.03488465043158973
  (0, 22)	0.03488465043158973
  (0, 21)	0.03488465043158973
  (0, 20)	0.03488465043158973
  (0, 2067)	0.1855277724508733
  (0, 6550)	0.12168612952324628
  (0, 11227)	0.20863451098655908
  (0, 8840)	0.17249080613467538
  (0, 7504)	0.24392385516674034
  (0, 12577)	0.16273758595019577
  (0, 14302)	0.21314201707089955
  (0, 12144)	0.22122833534676759
  (0, 11500)	0.2338881666338247
  (0, 14831)	0.2169513961984754
  (0, 14572)	0.253959543699656
  (0, 11527)	0.16162118186339838
  (0, 12309)	0.2338881666338247
  (0, 2761)	0.26810405225821604
  :	:
  (4802, 3099)	0.2590224858071576
  (4802, 2062)	0.23683206350769068
  (4802, 3901)	0.23683206350769068
  (4802, 5407)	0.21464164120822374
  (4802, 3687)	0.15177861546596524
  (4802, 11195)	0.1673919518

In [47]:
similarity = cosine_similarity(ftr_vector)
print(similarity)

[[1.         0.03828405 0.05798493 ... 0.01232936 0.01466137 0.01228477]
 [0.03828405 1.         0.0267995  ... 0.02701016 0.01266239 0.01060982]
 [0.05798493 0.0267995  1.         ... 0.01272265 0.07989281 0.01267664]
 ...
 [0.01232936 0.02701016 0.01272265 ... 1.         0.01350373 0.03657365]
 [0.01466137 0.01266239 0.07989281 ... 0.01350373 1.         0.0134549 ]
 [0.01228477 0.01060982 0.01267664 ... 0.03657365 0.0134549  1.        ]]


In [48]:
print(similarity.shape)

(4803, 4803)


In [49]:
enter_movie_of_choice = input("Search for a movie...")

Search for a movie...iron man


In [50]:
movies_list = movie_dataset['title'].tolist()
print(movies_list)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [51]:
list_of_close_match_titles = difflib.get_close_matches(enter_movie_of_choice, movies_list)
print(list_of_close_match_titles)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [52]:
first_match = list_of_close_match_titles[0]
print(first_match)

Iron Man


In [53]:
index = movie_dataset[movie_dataset.title == first_match]['index'].values[0]
print(index)

68


In [54]:
calculated_similarity = list(enumerate(similarity[index]))
print(calculated_similarity)

[(0, 0.05387716151501022), (1, 0.07099776820604134), (2, 0.030443730778660964), (3, 0.021939360996729638), (4, 0.054311631697431424), (5, 0.029110773012587375), (6, 0.048754313910210685), (7, 0.29410519827063025), (8, 0.023181719618925116), (9, 0.1043245896632827), (10, 0.09665675713704791), (11, 0.02859776923530941), (12, 0.028740682390984165), (13, 0.02684049050764279), (14, 0.13115159267184218), (15, 0.020657364941016384), (16, 0.2910804208041512), (17, 0.02951573078224282), (18, 0.0411458006215544), (19, 0.07927228187193659), (20, 0.1077798485245122), (21, 0.028313027885081096), (22, 0.023691128213594755), (23, 0.023395379096191597), (24, 0.027653722087614737), (25, 0.013786257119275225), (26, 0.2741318385889121), (27, 0.04839661864003971), (28, 0.08565779595986601), (29, 0.03069070573596588), (30, 0.10794025179980636), (31, 0.4070554041715449), (32, 0.021750960459491167), (33, 0.16324376448404643), (34, 0.013698629709992399), (35, 0.05437630495289167), (36, 0.05096652247231122), (

In [55]:
len(calculated_similarity)

4803

In [56]:
sorted_calculated_similarity_list = sorted(calculated_similarity, key = lambda x : x[1], reverse = True)
print(sorted_calculated_similarity_list)

[(68, 0.9999999999999998), (79, 0.5495575162537538), (31, 0.4070554041715449), (7, 0.29410519827063025), (16, 0.2910804208041512), (26, 0.2741318385889121), (3623, 0.21220286024165003), (85, 0.21105258790632522), (511, 0.2092980165261583), (203, 0.20862323048262996), (182, 0.20567928005437805), (101, 0.20170826350259197), (4401, 0.19837963180618864), (64, 0.1966648485058123), (174, 0.1931391559934131), (2487, 0.18708325689342034), (788, 0.17683956321994007), (1740, 0.17464220885256282), (46, 0.173237868971668), (126, 0.17289621383739145), (1553, 0.1650482158576112), (33, 0.16324376448404643), (2442, 0.15831607420635274), (783, 0.1579845037707377), (2651, 0.15669406098998354), (138, 0.15541967122981137), (607, 0.1470126268700645), (307, 0.14515212891160564), (38, 0.14503443958488133), (122, 0.1445664103840914), (1368, 0.14375339534027132), (661, 0.1419397269311331), (242, 0.13735784016441122), (2235, 0.13729336128466585), (2390, 0.1363996352005818), (954, 0.13615129420738437), (169, 0.1

In [57]:
print("Movies Recommended based on your search...")
i = 1
for movie_title in sorted_calculated_similarity_list:
    index = movie_title[0]
    title_from_index = movie_dataset[movie_dataset.index == index]['title'].values[0]
    if i < 30:
        print(i, '.', title_from_index)
        i += 1

Movies Recommended based on your search...
1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Made
8 . Captain America: The Winter Soldier
9 . X-Men
10 . X2
11 . Ant-Man
12 . X-Men: First Class
13 . The Helix... Loaded
14 . X-Men: Apocalypse
15 . The Incredible Hulk
16 . Duets
17 . Deadpool
18 . Kick-Ass 2
19 . X-Men: Days of Future Past
20 . Thor: The Dark World
21 . Se7en
22 . X-Men: The Last Stand
23 . Southland Tales
24 . Mortdecai
25 . The Good Night
26 . The Last Airbender
27 . Sky Captain and the World of Tomorrow
28 . The Expendables 3
29 . The Amazing Spider-Man 2


### Movie Recommendation System

In [58]:
enter_movie_of_choice = input("Search for a hollywood movie...")
movies_list = movie_dataset['title'].tolist()
list_of_close_match_titles = difflib.get_close_matches(enter_movie_of_choice, movies_list)
if enter_movie_of_choice not in list_of_close_match_titles:
    print("Match not found.... Check if it is a hollywood film.... ")
first_match = list_of_close_match_titles[0]
index = movie_dataset[movie_dataset.title == first_match]['index'].values[0]
calculated_similarity = list(enumerate(similarity[index]))
sorted_calculated_similarity_list = sorted(calculated_similarity, key = lambda x : x[1], reverse = True)
print("Movies Recommended based on your search...")
i = 1
for movie_title in sorted_calculated_similarity_list:
    index = movie_title[0]
    title_from_index = movie_dataset[movie_dataset.index == index]['title'].values[0]
    id_from_index = movie_dataset[movie_dataset.index == index]['id'].values[0]
    if i < 30:
        print(i, '.', title_from_index, id_from_index)
        i += 1

Search for a hollywood movie...made
Match not found.... Check if it is a hollywood film.... 
Movies Recommended based on your search...
1 . Made 15745
2 . Couples Retreat 19899
3 . The Break-Up 9767
4 . Identity Thief 109431
5 . Swingers 10218
6 . Very Bad Things 10029
7 . Undisputed 15070
8 . G-Force 19585
9 . Four Christmases 12193
10 . Zathura: A Space Adventure 6795
11 . Iron Man 1726
12 . Iron Man 2 10138
13 . Zookeeper 38317
14 . The Jungle Book 278927
15 . Elf 10719
16 . Corky Romano 17708
17 . House on Haunted Hill 11377
18 . GoldenEye 710
19 . National Treasure 2059
20 . I Spy 8427
21 . Taken 3 260346
22 . The Dilemma 44564
23 . The Benchwarmers 9957
24 . Taken 8681
25 . The Wackness 13990
26 . Mr. & Mrs. Smith 787
27 . Hide and Seek 11096
28 . X2 36658
29 . Unfinished Business 239573


In [27]:
pickle.dump(movie_dataset.to_dict(), open('movies_dict.pkl','wb'))

pickle.dump(similarity, open('similarity.pkl','wb'))